# RC Setpoint generation

We aim for acro mode control, where pilot inputs command rates. The most straight forward solution to multiply
the normalized (-1.0 to 1.0) input by the maximum rate per axis. Additionally some math is applied to optimize
it for better handling/feeling (expo).

There are multiple variants available in open source flight controller codes. In the following the px4 variant
is reimplemented and visualized.

In [1]:
import numpy as np

In [2]:
def px4_acro(pilot_in, e, g):
    # clip inputs
    x = np.clip(pilot_in, -0.999, 0.999)
    e = np.clip(e, 0.0, 0.99)
    g = np.clip(g, 0.00, 1.0)
    
    # apply expo/superexpo
    expo = (1 - e) * x + e * x * x * x
    superexpo = expo * (1 - g) / (1 - abs(x) * g)
    
    return superexpo
    

## Visualization

In [3]:
input_range = np.linspace(-1.0, 1.0, 1000)

def plot_setpoint(pilot_in, e, g):

    outputs = [px4_acro(x, e, g) for x in input_range]
    
    y = px4_acro(pilot_in, e, g)
    
    plt.figure(2)
    
    plt.plot(input_range, outputs)
    plt.plot(pilot_in, y, "ro")
    
    plt.show()

In [4]:
%matplotlib inline
from matplotlib import pyplot as plt
from ipywidgets import Layout, Button, Box, IntSlider, FloatSlider, Output, Label, interactive_output

input_slider_options = {'value': 0.0, 'min':-1.0, 'max':1.0, 'step':0.01}
input_labels = ['input', 'e', 'g']
inputs = {
    "pilot_in": FloatSlider(description="input", value=0.0, min=-1.0, max=1.0, step=0.01),
    "e": FloatSlider(description="e", value=0.0, min=0.0, max=1.0, step=0.01),
    "g": FloatSlider(description="g", value=0.0, min=0.0, max=0.99, step=0.01),
}
items = [Label(value='Controls'), *inputs.values()]

vbox_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='400px')

vbox = Box(children=items, layout=vbox_layout)

hbox_layout = Layout(display='flex',
                     flex_flow='row',
                     align_items='stretch',
                     width='100%')

out1 = interactive_output(plot_setpoint, inputs)


hbox = Box(children=(vbox, out1), layout=hbox_layout)

hbox

Box(children=(Box(children=(Label(value='Controls'), FloatSlider(value=0.0, description='input', max=1.0, min=…

## Test data

Data generation for C/C++ implementation tests,

In [5]:
from itertools import product

In [6]:
pilot_input_values = [-1.1, -1.0, -0.3, 0.0, 0.88, 0.999, 1.0, 1.5]
e_input_values = [-0.001, 0.0, 0.4, 0.99, 1.0]
g_input_values = [-0.9, 0.0, 0.1, 0.5, 1.0, 2.9]

data = []

for p_in, e_in, g_in in product(pilot_input_values, e_input_values, g_input_values):
    
    res = px4_acro(p_in, e_in, g_in)
    
    data.append([p_in, e_in, g_in, res])
    

In [7]:
data = np.array(data)

with open("../../tests/drivers/rc/src/test_rate_setpoint.inc", "w") as afile:
    afile.write("// This file is autogenerated by 03_SetpointGeneration, do not alter\n\n")
    afile.write("// Test Data columns: pilot_in, e, g, max_rate, output\n")
    afile.write(f"const float test_data_setpoint[{data.shape[0]}][{data.shape[1]}] = {{\n")
    for row in data:
        afile.write(("\t{{" + len(row)*"{:2.16f}f, " +"}},\n").format(*row))

    afile.write("};\n")